# and then we go to mario draghi

In [1]:
import pandas as pd
import time
import datetime
import matplotlib
import numpy as np
from statsmodels.distributions.empirical_distribution import ECDF

ad = pd.read_csv('datasets/1976-2020-president.csv')
years = set(ad['year'])
states = set(ad['state'])

#### some function

In [2]:
def ecdf_inv(a, q): return sorted(a)[int(len(a) * q)]

In [3]:
def to_percent(older, col_key):
    newer = []
    for y in years:
        for s in states:
            try:
                d = older.loc[y, s, 'DEMOCRAT'][col_key]
                r = older.loc[y, s, 'REPUBLICAN'][col_key]
                newer.append([int(y), s, 100*d/(d+r)]) # % share for each state
            except KeyError as e:
                pass
    return pd.DataFrame(data=newer, columns=['year', 'state', col_key, ])

In [4]:
def to_splatter(a, col_name): # from row-column to table-list
    lst = []
    for year in a.columns[2:]:
        for index, row in a.iterrows():
            lst.append([int(year), row['GeoName'].upper(), row[year], ])
    
    return pd.DataFrame(data=lst, columns=['year', 'state', col_name, ])

#### y_hat aka presitdent popular vote

In [5]:
pop_vote = pd.read_csv('datasets/1976-2020-president.csv')
pop_vote = pop_vote[['year', 'state', 'party_simplified', 'candidatevotes', ]] # drop useless columns
pop_vote = pop_vote.groupby(['year', 'state', 'party_simplified', ]).sum() # drop multiple candidate
pop_vote = to_percent(pop_vote, 'candidatevotes')
pop_vote = pop_vote.rename(columns={"candidatevotes": "y_votes_percent", })
pop_vote = pop_vote.sort_values(by=['year', 'state'])
pop_vote.to_csv('datasets-clean/popular-vote-y.csv', index=False)
pop_vote

,year,state,y_votes_percent
560,1976,ALABAMA,56.666724
556,1976,ALASKA,38.108171
535,1976,ARIZONA,41.386697
510,1976,ARKANSAS,65.048851
533,1976,CALIFORNIA,49.082173
...,...,...,...
163,2020,VIRGINIA,55.154687
187,2020,WASHINGTON,59.925503
170,2020,WEST VIRGINIA,30.201468
175,2020,WISCONSIN,50.319063


#### GDP and friends

It was necessary to generate an adjusted series of state GDP because of a change in BEA’s estimation procedure from a Standard Industrial Classification (SIC) basis to a North American Industry Classification System (NAICS) basis in 1997.
Data prior to 1997 were adjusted to avoid any erratic shifts in GDP that year.
While the change to NAICS basis occurred in 1997, BEA also provides estimates under a SIC basis in that year.
Our adjustment involved calculating the 1997 ratio of NAICS-based GDP to SIC-based GDP for each state, and multiplying it by SIC-based GDP in all years prior to 1997 to obtain our adjusted series of state-level GDP.

In [6]:
gdp_nom_97 = to_splatter(pd.read_csv('datasets/gdp-nominal-63-97.csv'), 'nominal_gdp_mln')
gdp_nom_12 = to_splatter(pd.read_csv('datasets/gdp-nominal-97-20.csv'), 'nominal_gdp_mln')
gdp_real_97 = to_splatter(pd.read_csv('datasets/gdp-real-77-97-chain-97.csv'), 'real_gdp_mln')
gdp_real_12 = to_splatter(pd.read_csv('datasets/gdp-real-97-20-chain-12.csv'), 'real_gdp_mln')
gdp_nom_97 = gdp_nom_97.set_index(['year', 'state'])
gdp_nom_12 = gdp_nom_12.set_index(['year', 'state'])
gdp_real_97 = gdp_real_97.set_index(['year', 'state'])
gdp_real_12 = gdp_real_12.set_index(['year', 'state'])

In [7]:
gdp_def_97 = (gdp_nom_97['nominal_gdp_mln'] / gdp_real_97['real_gdp_mln']).to_frame('gdp_def').dropna()
gdp_def_12 = (gdp_nom_12['nominal_gdp_mln'] / gdp_real_12['real_gdp_mln']).to_frame('gdp_def').dropna()

In [8]:
gdp_def = []

for s in states:
    tmp = gdp_def_97.loc[(slice(None),s),:]
    yyyy = list({i[0] for i in tmp.index})
    yyyy.sort()
    for y in yyyy[1:]:
        q = gdp_def_97.loc[(y,s),:]['gdp_def']/gdp_def_97.loc[(int(y)-1,s),:]['gdp_def']
        gdp_def.append([y, s, (q - 1)*100])

for s in states:
    tmp = gdp_def_12.loc[(slice(None),s),:]
    yyyy = list({i[0] for i in tmp.index})
    yyyy.sort()
    for y in yyyy[1:]:
        q = gdp_def_12.loc[(y,s),:]['gdp_def']/gdp_def_12.loc[(int(y)-1,s),:]['gdp_def']
        gdp_def.append([y, s, (q - 1)*100])

gdp_def = pd.DataFrame(data=gdp_def, columns=['year', 'state', 'gdp_def'])
gdp_def = gdp_def.sort_values(by=['year', 'state'])
gdp_def.to_csv('datasets-clean/gdp-def-percent-change-year-ago.csv', index=False)
gdp_def

,year,state,gdp_def
1000,1978,ALABAMA,7.402566
920,1978,ALASKA,11.065835
500,1978,ARIZONA,7.468516
0,1978,ARKANSAS,8.054511
460,1978,CALIFORNIA,7.094885
...,...,...,...
1272,2020,VIRGINIA,1.950355
1824,2020,WASHINGTON,1.658615
1433,2020,WEST VIRGINIA,-0.257632
1548,2020,WISCONSIN,2.055099


In [9]:
gdp_real = []

for s in states:
    tmp = gdp_real_97.loc[(slice(None),s),:]
    yyyy = list({i[0] for i in tmp.index})
    yyyy.sort()
    for y in yyyy[1:]:
        q = gdp_real_97.loc[(y,s),:]['real_gdp_mln']/gdp_real_97.loc[(int(y)-1,s),:]['real_gdp_mln']
        gdp_real.append([y, s, (q - 1)*100])

for s in states:
    tmp = gdp_real_12.loc[(slice(None),s),:]
    yyyy = list({i[0] for i in tmp.index})
    yyyy.sort()
    for y in yyyy[1:]:
        q = gdp_real_12.loc[(y,s),:]['real_gdp_mln']/gdp_real_12.loc[(int(y)-1,s),:]['real_gdp_mln']
        gdp_real.append([y, s, (q - 1)*100])

gdp_real = pd.DataFrame(data=gdp_real, columns=['year', 'state', 'gdp_real'])
gdp_real = gdp_real.sort_values(by=['year', 'state'])
gdp_real.to_csv('datasets-clean/gdp-real-percent-change-year-ago.csv', index=False)
gdp_real

,year,state,gdp_real
1000,1978,ALABAMA,6.421827
920,1978,ALASKA,8.958679
500,1978,ARIZONA,10.462245
0,1978,ARKANSAS,6.534163
460,1978,CALIFORNIA,6.881903
...,...,...,...
1272,2020,VIRGINIA,-2.757118
1824,2020,WASHINGTON,-0.581850
1433,2020,WEST VIRGINIA,-3.902932
1548,2020,WISCONSIN,-4.006500


In [10]:
z_yearly = gdp_real.copy()
z_yearly = z_yearly.set_index(['year', 'state'])
z_yearly.gdp_real = z_yearly.gdp_real.apply(lambda x : 1 if x > 2.67 else 0)
z_yearly

gdp_real
year state                  
1978 ALABAMA               1
     ALASKA                1
     ARIZONA               1
     ARKANSAS              1
     CALIFORNIA            1
...                      ...
2020 VIRGINIA              0
     WASHINGTON            0
     WEST VIRGINIA         0
     WISCONSIN             0
     WYOMING               0

[2193 rows x 1 columns]

In [12]:
growth_index = []
for y in years:
    for s in states:
        try:
            growth_index.append([y,s,] + [z_yearly.loc[y-i,s].gdp_real  for i in range(4)])
        except KeyError:
            pass

growth_index = pd.DataFrame(data=growth_index, columns=['year', 'state', 'z_t_0', 'z_t_1', 'z_t_2', 'z_t_3',])
growth_index = growth_index.sort_values(by=['year', 'state'])
growth_index.to_csv('datasets-clean/z-growth-index.csv', index=False)
growth_index

,year,state,z_t_0,z_t_1,z_t_2,z_t_3
50,1984,ALABAMA,1,1,0,0
46,1984,ALASKA,1,0,0,1
25,1984,ARIZONA,1,1,0,1
0,1984,ARKANSAS,1,1,0,1
23,1984,CALIFORNIA,1,1,0,1
...,...,...,...,...,...,...
163,2020,VIRGINIA,0,0,0,0
187,2020,WASHINGTON,0,1,1,1
170,2020,WEST VIRGINIA,0,0,0,0
175,2020,WISCONSIN,0,0,0,0


us stuff

In [ ]:
gdp_us = pd.read_csv('datasets/gdp-nomina-47-20-chain-12.csv')
gdp_us.DATE = gdp_us.DATE.apply(lambda x: int(str(x).split('-')[0]))
s = gdp_us[gdp_us.DATE > 1975].GDPC1_PC1
s.hist()

In [ ]:
ecdf = ECDF(s)
ecdf(2.8)

qnt = 0.67
std_err = np.sqrt(s.var())/2
ecdf_inv(s, qnt), len(s)*(1-qnt), std_err

#### house

In [ ]:
house_vote = pd.read_csv('datasets/1976-2020-house-utf8.csv')
house_vote = house_vote[['year', 'state', 'party', 'candidatevotes', ]]
house_vote = house_vote.groupby(['year', 'state', 'party', ]).sum()
house_vote = to_percent(house_vote, 'candidatevotes')
house_vote['candidatevotes'] = house_vote['candidatevotes'].apply(lambda x: 1 if x < 50 else -1)
house_vote['year'] -= 4
house_vote = house_vote.rename(columns={'candidatevotes':'incumbent'})
house_vote.to_csv('datasets-clean/incumbent-house-rep.csv', index=False)
house_vote.head()

In [ ]:
'''
house_vote = pd.read_csv('datasets/1976-2020-house-utf8.csv')
house_vote = house_vote[['year', 'state', 'party', 'candidatevotes', ]]
house_vote = house_vote.groupby(['year', 'state', 'party', ]).sum()
house_vote = to_percent(house_vote, 'candidatevotes')
house_vote = house_vote.rename(columns={"candidatevotes": "houserep_votes_percent", })
#'''

#### incumbent

In [ ]:
incumbent = pd.read_csv('datasets/incumbent-4president-76-20.csv', sep=';')
incumbent.head()

tmp = pd.DataFrame(data=[[y,s] for y in years for s in states], columns=['year', 'state'])
incumbent = pd.merge(tmp, incumbent, how='inner', left_on='year', right_on='year')
incumbent.head()

#### gasoline and friends + approval

In [ ]:
gas = pd.read_csv('datasets/gasoline-93-21.csv')
gas.date = gas.date.apply(lambda x: time.mktime(datetime.datetime.strptime(x,"%m/%d/%Y").timetuple()))
gas.date = gas.date.apply(lambda x: datetime.datetime.fromtimestamp(int(x)))
gas.head()

#### houses prices and rent + personal income

In [ ]:
f = open('datasets/house-chain-00.csv', 'rt')
lines = f.readlines()
houses = [[i.strip() for i in l.split(sep = '$') ] for l in lines ]
houses = pd.DataFrame(data=houses, columns=['state', 2000, 1990, 1980, 1970, 1960, 1950, 1940 ])
houses.head()

#### merging features

In [ ]:
'''
df = pd.merge(pop_vote, nom_gdp, how='inner', left_on=['year','state'], right_on=['year','state'])
df = pd.merge(df, house_vote, how='inner', left_on=['year','state'], right_on=['year','state'])
df.head()
#'''

In [ ]:
'''
df.to_csv('datasets-clean/xxx-final-dataset.csv', index=False)
#'''